In [ ]:
from fake_useragent import UserAgent
from bs4 import BeautifulSoup
import time
import numpy as np
import requests
import json
import csv
import io

In [ ]:
def commentSave(list_comment):
    file = io.open('JD_Review_data.txt','w',encoding="utf-8",newline = '')
    writer = csv.writer(file)
    writer.writerow(['ID','Review','Purchase date','Vote','Reply','Score','Review date','Model'])
    for i in range(len(list_comment)):
        writer.writerow(list_comment[i])
    file.close()
    print('Data Saved')

def getCommentData(format_url,proc,i,maxPage):
    sig_comment = []
    global list_comment
    cur_page = 0
    while cur_page < maxPage:
        cur_page += 1
        # url = 'https://sclub.jd.com/comment/productPageComments.action?callback=fetchJSON_comment98vv%s&score=%s&sortType=5&page=%s&pageSize=10&isShadowSku=0&fold=1'%(proc,i,cur_page)
        url = format_url.format(proc,i,cur_page) 
        try:
            response = requests.get(url=url, headers=headers)
            time.sleep(np.random.rand()*2)
            jsonData = response.text
            startLoc = jsonData.find('{')
            #print(jsonData[::-1])
            jsonData = jsonData[startLoc:-2]
            jsonData = json.loads(jsonData)
            pageLen = len(jsonData['comments'])
            print("Page %s"%cur_page)
            for j in range(0,pageLen):
                userId = jsonData['comments'][j]['id']
                content = jsonData['comments'][j]['content']
                boughtTime = jsonData['comments'][j]['referenceTime']
                voteCount = jsonData['comments'][j]['usefulVoteCount']
                replyCount = jsonData['comments'][j]['replyCount']
                starStep = jsonData['comments'][j]['score']
                creationTime = jsonData['comments'][j]['creationTime']
                referenceName = jsonData['comments'][j]['referenceName']
                sig_comment.append(userId)
                sig_comment.append(content)
                sig_comment.append(boughtTime)
                sig_comment.append(voteCount)
                sig_comment.append(replyCount)
                sig_comment.append(starStep)
                sig_comment.append(creationTime)
                sig_comment.append(referenceName)
                list_comment.append(sig_comment)
                sig_comment = []
        except:
            time.sleep(5)
            cur_page -= 1
            print('Network error, retry later')

In [ ]:
if __name__ == "__main__":
    global list_comment
    ua=UserAgent()
    format_url = 'https://club.jd.com/comment/productPageComments.action?callback=fetchJSON_comment98&{0}&score={1}&sortType=5&page={2}&pageSize=10&isShadowSku=0&fold=1'
    headers = {
    'Accept': '*/*',
    "User-Agent":ua.random,
    'Referer':"https://item.jd.com/100000177760.html#comment"
    }
    productid = ['productId=100018276690'] # Change to product id for scraping
    list_comment = [[]]
    sig_comment = []
    for proc in productid:
        i = 0
        while i < 3: # change to score level: 1:lowest, 5:highest 
            i += 1
            if(i == 6):
                continue
            url = format_url.format(proc,i,0)
            print(url)
            try:
                response = requests.get(url=url, headers=headers)
                jsonData = response.text
                startLoc = jsonData.find('{')
                jsonData = jsonData[startLoc:-2]
                jsonData = json.loads(jsonData)
                print("Max page %s"%jsonData['maxPage'])
                getCommentData(format_url,proc,i,jsonData['maxPage'])
            except Exception as e:
                i -= 1
                print("the error is ",e)
                print("wating---")
                time.sleep(5)
                #commentSave(list_comment)
    print("Scrape complete, saving data")
    commentSave(list_comment)